# 기본 ConvLSTM2D 

참고 자료 : https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

ConvLSTM2D class, by default, expects input data to have the shape :   
`(samples, time, rows, cols, channels)`

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os , glob

from tqdm import tqdm
from pathlib import Path
import datetime
import time

import tensorflow as tf
from tensorflow import keras

# convlsm model 
from keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from keras.layers import ConvLSTM2D

In [3]:
def sorted_list(path):
    tmplist = glob.glob(path)
    tmplist.sort()
    
    return tmplist 

In [4]:
list_train = sorted_list(os.path.join('data/train','*'))
len(list_train)

482

In [5]:
filenames = os.listdir('data/train')
filenames.sort()

In [6]:
data = []

for filename in tqdm(filenames):
    data.append(
        np.load(f"data/train/{filename}"))
    
data = np.array(data)

100%|██████████| 482/482 [00:02<00:00, 198.50it/s]


In [6]:
data.shape

(482, 448, 304, 5)

In [7]:
df_train = pd.read_csv('data/train.csv')

In [8]:
df_train.head(3)

,month,file_nm
0,1978-11,197811.npy
1,1978-12,197812.npy
2,1979-01,197901.npy


## 데이터 창 작업 
참고 자료 :https://www.tensorflow.org/tutorials/structured_data/time_series?hl=ko


window_size = 4    
offset = 24  # 24개월     
label width = 1    
target = 1    
window_stride = 24   
window_shfit = 1   

In [164]:
data[...,3]+=data[...,2] # concat masks
data.shape

(482, 448, 304, 5)

In [165]:
tr_npy = data[:-48, ..., [0, 3]]
ts_npy = data[-48:, ..., [0, 3]]
print(tr_npy.shape, ts_npy.shape)

(434, 448, 304, 2) (48, 448, 304, 2)


In [166]:
split_len = int(tr_npy.shape[0] * 0.25)  # validation set : 25% 
vl_npy = tr_npy[-split_len:]
tr_npy = tr_npy[:-split_len]

In [167]:
print(tr_npy.shape, vl_npy.shape, ts_npy.shape)

(326, 448, 304, 2) (108, 448, 304, 2) (48, 448, 304, 2)


In [336]:
@tf.function
def split_mask(x):
    return tf.split(x, [1,1], axis = -1)

@tf.function
def rescaling(images, masks):
    return (tf.cast(images, dtype = tf.dtypes.float32) / 250.,

            tf.cast(masks[:,0], tf.dtypes.float32))

@tf.function
def split_window(images, masks):
    inputs, target = tf.split(images, [4,1], axis =1)
    return (inputs, masks), target

In [ ]:
tr_ten = tf.constant(tr_npy)
vl_ten = tf.constant(vl_npy)
ts_ten = tf.constant(ts_npy)

tr_ds = tf.data.Dataset.from_tensor_slices(tr_ten
                                    ).window(4 + 1, shift = 1, stride = 24, drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4+1)
                                    ).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(split_mask
                                    ).map(rescaling
                                    ).map(split_window)

vl_ds = tf.data.Dataset.from_tensor_slices(vl_ten
                                    ).window(4 + 1, shift = 1, stride =  24, drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4+1)
                                    #).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(split_mask
                                    ).map(rescaling
                                    ).map(split_window)


ts_ds = tf.data.Dataset.from_tensor_slices(vl_ten
                                    ).window(4 , shift = 1, stride = 24, drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4)
                                    #).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(split_mask
                                    ).map(rescaling)
                                    #).map(split_window)

In [350]:
tr_ds.element_spec

((TensorSpec(shape=(None, 4, 448, 304, 1), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 448, 304, 1), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None))

In [351]:
vl_ds.element_spec

((TensorSpec(shape=(None, 4, 448, 304, 1), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 448, 304, 1), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None))

In [352]:
ts_ds.element_spec

(TensorSpec(shape=(None, None, 448, 304, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 448, 304, 1), dtype=tf.float32, name=None))

In [353]:
for ele in tr_ds.take(1):
    (foo, bar), qux = ele
    print(foo.shape, bar.shape, qux.shape)

(8, 4, 448, 304, 1) (8, 448, 304, 1) (8, 1, 448, 304, 1)


## Only 해빙 농도 channel

In [7]:
ice_tr_npy = data[:-48, ..., [0]]
ice_ts_npy = data[-48:, ..., [0]]
print(ice_tr_npy.shape, ice_ts_npy.shape)

(434, 448, 304, 1) (48, 448, 304, 1)


In [8]:
split_len = int(ice_tr_npy.shape[0] * 0.25)  # validation set : 25% 
ice_vl_npy = ice_tr_npy[-split_len:]
ice_tr_npy = ice_tr_npy[:-split_len]
print(ice_tr_npy.shape, ice_vl_npy.shape,ice_ts_npy.shape)

(326, 448, 304, 1) (108, 448, 304, 1) (48, 448, 304, 1)


In [9]:
@tf.function
def rescaling(images):
    return (tf.cast(images, dtype = tf.dtypes.float32) / 250.)

@tf.function
def split_window(images):
    inputs, target = tf.split(images, [4,1], axis =1)
    return (inputs, target)

In [10]:
i_tr_ten = tf.constant(ice_tr_npy)
i_vl_ten = tf.constant(ice_vl_npy)
i_ts_ten = tf.constant(ice_ts_npy)

i_tr_ds = tf.data.Dataset.from_tensor_slices(i_tr_ten
                                    ).window(4 + 1, shift = 1, stride = 24,drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4+1)
                                    ).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(rescaling
                                    ).map(split_window
                                    ).prefetch(tf.data.experimental.AUTOTUNE)

i_vl_ds = tf.data.Dataset.from_tensor_slices(i_vl_ten
                                    ).window(4 + 1, shift = 1, stride = 24,drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4+1)
                                    #).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(rescaling
                                    ).map(split_window
                                    ).prefetch(tf.data.experimental.AUTOTUNE)


i_ts_ds = tf.data.Dataset.from_tensor_slices(i_vl_ten
                                    ).window(4 , shift = 1, stride = 24,drop_remainder = True
                                    ).flat_map(lambda x : x.batch(4)
                                    #).shuffle(buffer_size = 1000
                                    ).batch(8
                                    ).map(rescaling
                                    ).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
print(f'{i_tr_ds.element_spec} \n',
      f'{i_vl_ds.element_spec} \n',
      f'{i_ts_ds.element_spec}')

(TensorSpec(shape=(None, 4, 448, 304, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None)) 
 (TensorSpec(shape=(None, 4, 448, 304, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None)) 
 TensorSpec(shape=(None, None, 448, 304, 1), dtype=tf.float32, name=None)


In [12]:
for inp, tag in i_tr_ds.take(1):
    print(f'input shape : {inp.shape}')
    print(f'label shape : {tag.shape}')

input shape : (8, 4, 448, 304, 1)
label shape : (8, 1, 448, 304, 1)


### loss function 

In [13]:
@tf.function
def mae_score(y_true, y_pred):
    return tf.math.reduce_mean(tf.math.abs(y_true - y_pred))


@tf.function
def f1_score(y_true, y_pred, lower_bound = 0.05, upper_bound = 0.5,
            threshold = 0.3, epsilon = 1e-8): 
    
    y_true = tf.where(y_true > upper_bound, 0., y_true)
    y_true = tf.where(y_true < lower_bound, 0., y_true)
    y_pred = tf.where(y_pred > upper_bound, 0., y_pred)
    y_pred = tf.where(y_pred < lower_bound, 0., y_pred)
    
    y_true = tf.where(y_true < 0.15, 0., 1.)
    y_pred = tf.where(y_pred < 0.15, 0., 1.)
    
    
    tp = tf.math.reduce_sum(tf.where(y_true*y_pred == 1., 1., 0.))
    precision = tp /(tf.math.reduce_sum(y_true) + epsilon)
    recall = tp / (tf.math.reduce_sum(y_pred)+epsilon)
    
    return 2*precision*recall / (precision + recall+ precision)

@tf.function
def mae_over_f1(y_true, y_pred, epsilon = 1e-8):
    return tf.math.divide_no_nan(mae_score(y_true, y_pred),
                                f1_score(y_true, y_pred)+ epsilon)


In [ ]:
model = Sequential()
model.add(layers.ConvLSTM2D(filters = 10, 
                    input_shape = (4,488,304,1),
                    kernel_size= (3,3),data_format = 'channels_last',
                    padding ='same', return_sequences=True,
                    recurrent_dropout = 0.4))

model.add(TimeDistributed(Flatten()))
model.add(Dense(488*304, activation ='sigmoid'))
model.add(Reshape((-1, 488,304,1)))


model.compile(optimizer ='adam', loss = mae_over_f1, 
             metrics = [mae_score, f1_score])

In [49]:
x.shape

(322, 4, 448, 304, 1)

In [15]:
y = np.concatenate([y for x, y in i_tr_ds], axis=0)

In [16]:
x = np.concatenate([x for x, y in i_tr_ds], axis=0)

In [17]:
val_df = (np.concatenate([x for x, y in i_vl_ds], axis = 0), 
         np.concatenate([y for x, y in i_vl_ds], axis = 0))

In [29]:
ice_tr_npy = data[:-48, ..., [0]]
ice_ts_npy = data[-48:, ..., [0]]


split_len = int(ice_tr_npy.shape[0] * 0.25)  # validation set : 25% 
ice_vl_npy = ice_tr_npy[-split_len:]
ice_tr_npy = ice_tr_npy[:-split_len]
print(ice_tr_npy.shape, ice_vl_npy.shape,ice_ts_npy.shape)

(326, 448, 304, 1) (108, 448, 304, 1) (48, 448, 304, 1)


In [66]:
class WindowGenerator(tf.keras.Model):
    def __init__(self, input_width, label_width , shift = 1, 
                 train = ice_tr_npy, val = ice_vl_npy, test = ice_ts_npy, label_columns = None):
        self.train_df = train
        self.val_df = val
        self.test_df = test
        
        # workout the label columns indices : 
        self.label_columns = label_columns 
        if label_columns is not None: 
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        
        
        
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        
        self.total_window_size = input_width + shift 
        
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
    def __repr__(self):
        return '\n'.join([
            f'toal window size : {self.total_window_size}',
            f'input indices : {self.input_indices}',
            f'label indices : {self.label_indices}'
        ])
    
    def split_window(self, images):
        inputs, target = tf.split(images, [1,1], axis =1)
       
        return inputs, target

    
    
    def make_dataset(self, data):
        data = np.array(data, dtype = np.float32)
        
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data = data , 
                targets = None, sequence_length = self.total_window_size,
                sequence_stride =1, 
                shuffle =True, batch_size = 8)
        
        ds = ds.map(self.split_window)
        
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
            
        
    @property 
    def example(self):
        result = getattr(self, '_example', None)
        
        if result is None:
            result = next(iter(self.train))
            self._example = result
            
        return result

In [67]:
wd = WindowGenerator(input_width = 1,label_width = 1, shift = 1)

In [68]:
wd.train.element_spec

(TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None))

In [69]:
for inputs, labels in wd.train.take(1):
    print(inputs.shape)

(8, 1, 448, 304, 1)


In [70]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index =None):
        super().__init__()
        
        self.label_index = label_index
        
    def call(self, inputs):
        if self.label_index is None :
            return inputs 
        
        resutl = inputs[:,:,self.label_index]
        return result[:,:,tf.newaxis]

In [75]:
# 단일 선형 모델 

def compile_and_fit(model, dataset, patience =2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor ='val_loss',
                                                     patience = patience,
                                                     mode ='min')
    
    model.compile(loss = tf.losses.MeanSquaredError(),
                 optimizer = tf.optimizers.Adam(),
                 metrics = [tf.metrics.MeanAbsoluteError()])
    
    history = model.fit(dataset.train, epochs = 30, 
                       validation_data = dataset.val,
                       callbacks = [early_stopping])
    
    
    return history

In [76]:
w2 = WindowGenerator(input_width = 24, label_width = 24, shift =1)
w2

toal window size : 25
input indices : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
label indices : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]

In [77]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units =1 )
])

history = compile_and_fit(linear, wd)

Epoch 1/30
41/41 [==============================] - 2s 46ms/step - loss: 3591.3772 - mean_absolute_error: 21.7579 - val_loss: 3068.6682 - val_mean_absolute_error: 19.4265
Epoch 2/30
41/41 [==============================] - 2s 40ms/step - loss: 3245.2053 - mean_absolute_error: 20.7475 - val_loss: 2777.2234 - val_mean_absolute_error: 18.5364
Epoch 3/30
41/41 [==============================] - 2s 45ms/step - loss: 2906.0427 - mean_absolute_error: 19.6574 - val_loss: 2511.0435 - val_mean_absolute_error: 17.6817
Epoch 4/30
41/41 [==============================] - 2s 49ms/step - loss: 2608.7399 - mean_absolute_error: 18.6283 - val_loss: 2269.2397 - val_mean_absolute_error: 16.8636
Epoch 5/30
41/41 [==============================] - 2s 41ms/step - loss: 2413.1470 - mean_absolute_error: 18.1473 - val_loss: 2050.6277 - val_mean_absolute_error: 16.0824
Epoch 6/30
41/41 [==============================] - 2s 42ms/step - loss: 2127.3171 - mean_absolute_error: 16.9718 - val_loss: 1853.7819 - val_mea

In [81]:
val_per= {}
perf = {}

In [82]:
val_per['Linear'] = linear.evaluate(wd.val)
perf['Linear'] = linear.evaluate(wd.test, verbose = 0)

14/14 [==============================] - 1s 27ms/step - loss: 526.7112 - mean_absolute_error: 6.9515


## 다중 스텝 모델 

In [103]:
label_width = 24
out_steps = 24 

multi_window = WindowGenerator(input_width = 24,
                             label_width = label_width,
                              shift = out_steps)

multi_window

toal window size : 48
input indices : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
label indices : [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]

In [96]:
wconv_window.train.element_spec

(TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 448, 304, 1), dtype=tf.float32, name=None))

In [101]:
class MultiStepLastBaseline(tf.keras.Model):
    
    def call(self, inputs):
        return tf.tile(input[:, -1:, :], [1, out_steps, 1])

In [ ]:
history = compile_and_fit(multi_step_dense, conv_window)

## 모델링

In [354]:
class convlstm(tf.keras.Model) :
    def __init__(self, name ):
        
        super(convlstm, self).__init__(name = name)
        
        self.convlstm = tf.keras.layers.ConvLSTM2D(256 , 3,  padding = 'same', 
                                                  return_state = True, recurrent_dropout = 0.4 )
        
        self.out_steps = 2
    
    def warmup(self, x):
        prediction, *state = self.convlstm(x)
        return prediction, state
        
    def call(self, inputs, traing =None):
        x, masks = inputs
        masks = tf.tile(tf.expand_dims(masks, axis = 1),[1,self.out_steps, 1,1,1])
        predictions = []
        
        prediction, state = self.warmup(x)
        predictions.append(prediction)
        
        for _ in range(1,self.out_steps):
                
            x ,*state = self.convlstm(tf.expand_dims(x, axis =1 ),
                                         inital_state = state)
            prediction = x 
                
            predictions.append(x)
                
        return tf.stack(predictions, axis = 1)
                

In [355]:
model_name = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
conv = convlstm(model_name)

In [356]:
conv.compile(optimizer = 'adam', loss = "mean_absolute_error",
             metrics =[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
_ = conv.fit(tr_ds, validation_data = vl_ds,
             epochs = 10,
             verbose = 2)